In [4]:
import pandas as pd
import torch
import esm
from tqdm.notebook import tqdm

In [3]:
# Load ESM-1b model
model, alphabet = esm.pretrained.esm1b_t33_650M_UR50S()
batch_converter = alphabet.get_batch_converter()
model = model.to("cuda")

In [5]:
def getSequenceRepresentation(Data):
    batch_labels, batch_strs, batch_tokens = batch_converter(Data)
    batch_tokens = batch_tokens.to("cuda")
    # Extract per-residue representations (on CPU)
    with torch.no_grad():
        results = model(batch_tokens, repr_layers=[33], return_contacts=True)
    token_representations = results["representations"][33].cpu()
    del results, batch_labels, batch_strs, batch_tokens
    # Generate per-sequence representations via averaging
    # NOTE: token 0 is always a beginning-of-sequence token, so the first residue is token 1.
    sequence_representations = []
    for i, (_, seq) in enumerate(Data):
        sequence_representations.append(token_representations[i, 1 : len(seq) + 1].cpu().numpy())
    del token_representations
    return sequence_representations#, results

In [6]:
df = pd.read_csv("/ssdata/clinvar/clinvar/parseClinvarWithVariant.csv")

In [8]:
df.head()

,Unnamed: 0,GeneID_x,variant,NM,label,idx,#AlleleID,Type,Name,GeneID_y,...,Guidelines,TestedInGTR,OtherIDs,SubmitterCategories,VariationID,PositionVCF,ReferenceAlleleVCF,AlternateAlleleVCF,nmid,variantSeq
0,0,57539,Ala864Thr,NM_020779.4,True,0,15062,single nucleotide variant,NM_020779.4(WDR35):c.2590G>A (p.Ala864Thr),57539,...,-,N,"ClinGen:CA339782,UniProtKB:Q9P2L0#VAR_064582,O...",3,23,20133230,C,T,NM_020779.4,MNCPVLSLGSGFLFQVIEMLIFAYFASISLTESRGLFPRLENVGAF...
1,2,112817,Gly287Val,NM_138413.4,True,1,15069,single nucleotide variant,NM_138413.4(HOGA1):c.860G>T (p.Gly287Val),112817,...,-,N,"ClinGen:CA113813,UniProtKB:Q86XE5#VAR_064036,O...",3,30,99371292,G,T,NM_138413.4,MNCPVLSLGSGFLFQVIEMLIFAYFASISLTESRGLFPRLENVGAF...
2,4,112817,Arg70Pro,NM_138413.4,True,2,15072,single nucleotide variant,NM_138413.4(HOGA1):c.209G>C (p.Arg70Pro),112817,...,-,N,"ClinGen:CA113816,OMIM:613597.0005",3,33,99344669,G,C,NM_138413.4,MNCPVLSLGSGFLFQVIEMLIFAYFASISLTESRGLFPRLENVGAF...
3,6,112817,Cys257Gly,NM_138413.4,True,3,15073,single nucleotide variant,NM_138413.4(HOGA1):c.769T>G (p.Cys257Gly),112817,...,-,N,"ClinGen:CA113818,UniProtKB:Q86XE5#VAR_064035,O...",3,34,99361682,T,G,NM_138413.4,MNCPVLSLGSGFLFQVIEMLIFAYFASISLTESRGLFPRLENVGAF...
4,8,22995,Gln265Pro,NM_001194998.2,True,4,15094,single nucleotide variant,NM_001194998.2(CEP152):c.794A>C (p.Gln265Pro),22995,...,-,N,"ClinGen:CA210936,UniProtKB:O94986#VAR_063813,O...",3,55,49085556,T,G,NM_001194998.2,MNCPVLSLGSGFLFQVIEMLIFAYFASISLTESRGLFPRLENVGAF...


In [12]:
def prepSeq(s,loc,windowSize=510):
    sPrime = s[max(0, loc-windowSize) : min(len(s), loc + windowSize)]
    return sPrime

In [14]:
representations = []
batchSize=1
for idx,row in tqdm(df.iterrows()):
    rep = getSequenceRepresentation([(idx,prepSeq(row.variantSeq, int(row.variant[3:-3])-1))])
    representations.append(rep)

RuntimeError: CUDA out of memory. Tried to allocate 2.57 GiB (GPU 0; 10.76 GiB total capacity; 5.10 GiB already allocated; 1.59 GiB free; 5.16 GiB reserved in total by PyTorch)